In [26]:
# !pip install pandas_datareader

In [27]:
import math
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pandas_datareader as pdr
from datetime import datetime, timedelta

In [28]:
#Get the stock
ticker = 'BAC'
stock ="Bank Of America"
today = datetime.today()

In [29]:
# for predictions add 100 extra days to make sure it has more than ts_points to make up for missing days (weekends and holidays)
# while calling thos model we will have to pass the excat number of ts_points as that were used while training the model
ts_points = 120
fetch_days = ts_points + 100    

sd =  today - timedelta(days=fetch_days)
sd = sd.strftime('%Y-%m-%d')

ed = today.strftime('%Y-%m-%d')
ed

next_day = (today + timedelta(days=1) ).strftime('%Y-%m-%d')
# next_day

In [30]:
# get the quote
quote = pdr.DataReader(ticker, data_source="yahoo", start=sd, end=ed)
new_df = quote.filter(['Close'])
last_n_days = ts_points
last_n_days_arr = new_df[-last_n_days:].values 

In [31]:
scaler = MinMaxScaler(feature_range=(0,1))
last_n_days_scaled = scaler.fit_transform(last_n_days_arr)
X_test = []
X_test.append(last_n_days_scaled)
X_test = np.array(X_test)
X_test = np.reshape(X_test,(X_test.shape[0], X_test.shape[1],1))


In [32]:
##########################loading saved model ###############################
new_model = tf.keras.models.load_model('Model_'+ticker+'.h5')
# Check its architecture
new_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 120, 50)           10400     
_________________________________________________________________
lstm_5 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 26        
Total params: 31,901
Trainable params: 31,901
Non-trainable params: 0
_________________________________________________________________


In [33]:
# Use The Model To Predict
nmpred_price = new_model.predict(X_test)
nmpred_price = scaler.inverse_transform(nmpred_price)
nmpred_price = nmpred_price[0][0]
print(f"Predicted Closing price for '{stock}' on {next_day} is {nmpred_price:.4f}")

Predicted Closing price for 'Bank Of America' on 2021-10-21 is 44.4667


In [34]:
# Look at last 4 days to see if it is close 
d1 = today - timedelta(days=5)
d1 = d1.strftime('%Y-%m-%d')

quote2 = pdr.DataReader(ticker, data_source="yahoo", start=d1)
quote2

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2021-10-15,46.669998,45.360001,45.770000,46.369999,64248200,46.369999
2021-10-18,46.889999,46.029999,46.290001,46.279999,49542500,46.279999
2021-10-19,46.630001,46.200001,46.500000,46.480000,32515500,46.480000
2021-10-20,47.240002,46.049999,46.490002,47.090000,39481326,47.090000
